In [19]:
##############
# Minimizers #
##############

# Note: All minimizer functions returns an array L such that
# L[1] = x^*, or the optimal point
# L[2] = value of the objective function at the optimal point
# L[3] = number of iterations used or DEFAULT_MAX_ITER

const DEFAULT_MAX_ITER = 500
const DEFAULT_TOLERANCE = 0.01
const DEFAULT_MAX_UPPER_BOUND = 1000


# Minimization algorithm using Newton's method
# f  the objective function
# g  the gradient of f
# h  the Hessian of f
# x  an initial testing point
# tolerance a small number
function minimizenewt(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    gamma =0.4
    iter = 0
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*norm(g(x))

    while norm(g(x)) > tolerance && iter < max_iter
        dx = -H(x)\g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
    end
    if iter < max_iter
        success =1
    else
        success =0
    end
    return [f(x), iter]
end





# Minimization algorithm using BFGS
# f  the objective function
# g  the gradient of f
# H  the initial guess of the INVERSE HESSIAN, you should set it to the identify matrix if you don't know
# x  an initial testing point
# tolerance a small number
function minimizeBFGS(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)

    alpha =0.3
    beta = 0.7
    gamma = 0.4
    iter = 0
    
    I = eye(length(x))
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*norm(g(x))

    while norm(g(x)) > tolerance && iter < max_iter
        dx = -H*g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end

        x1 = x + t*dx
        s = t*dx
        y = g(x1) - g(x)
        rho = 1/dot(s,y)
        H = (I-rho* s *(y'))*H*(I-rho*y*(s')) + rho*s*(s')
        x = x1
        iter = iter + 1
    end
    return [f(x), iter]
end




minimizeBFGS (generic function with 3 methods)

In [3]:
################################
# Simple test with a quadratic #
################################

m = 100
n = 2
x0 = [3;1]
A = randn(m,n)
b = A*x0
f0(x) = 0.5*norm(A*x-b)^2
g0(x) = A'*(A*x-b)
H0(x) = A'*A

println("Legends:")
println("NM = Newton's method")
println("BFGS = quasi-Newton method using BFGS approximation of inverse Hessian")
println("Correct = the correct outputs a minimizer function should spit out")
println("---------------------------------------------------------------------------------------")
@printf("%-10s %-40s %20s %20s\n", "Method", "Coordinate of optimal point", "Value of minimum", "# of iterations")
L = minimizenewt(f0,g0,H0,[0;0])
@printf("%-10s %10f %20f %20f %20i\n", "NM", L[1], L[2], L[3], L[4])
L = minimizeBFGS(f0,g0,eye(length(x0)), [0;0])
@printf("%-10s %10f %20f %20f %20i\n", "BFGS", L[1], L[2], L[3], L[4])
@printf("%-10s %10f %20f %20f %20i\n", "Correct", 3.0, 1.0, 0.0, 0)




Legends:


NM = Newton's method
BFGS = quasi-Newton method using BFGS approximation of inverse Hessian
Correct = the correct outputs a minimizer function should spit out
---------------------------------------------------------------------------------------
Method     Coordinate of optimal point                  Value of minimum      # of iterations


 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at ./abstractarray.jl:29
 in vect at abstractarray.jl:38
 in minimizenewt at In[2]:44
 in minimizenewt at In[2]:22
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /opt/julia_packages/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /opt/julia_packages/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[3], in expression starting on line 20


NM 

          3.000000             1.000000             0.000000                    1


 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at ./abstractarray.jl:29
 in vect at abstractarray.jl:38
 in minimizeBFGS at In[2]:84
 in minimizeBFGS at In[2]:59
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /opt/julia_packages/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /opt/julia_packages/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[3], in expression starting on line 22


BFGS         2.999999             1.000001             0.000000                    4
Correct 

In [4]:
Pkg.add("Toms566")
Pkg.update()
using Toms566

INFO: Nothing to be done


     3.000000             1.000000             0.000000                    0


INFO: Updating METADATA...
INFO: Updating Toms566...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [26]:

######################################################
# Complete test of every functions in Toms566, SLOW! #
######################################################

println("Legends:")
println("NM = Newton's method")
println("BFGS = quasi-Newton method using BFGS approximation of inverse Hessian")
println("---------------------------------------------------------------------------------------")
@printf("%3s  %-30s  %-30s", "", "", "# of iterations (-1 = failure)\n")
@printf("%3s  %-30s  %-6s %-20s %-8s %-20s \n",
        "No.",
        "Name",
        "NW", 
        "value",
        "BFGS",
        "value")
for i=1:18
    p = Problem(i)
    x0 = p.x0       # standard starting point
    f = p.obj   # objective value at x
    g = p.grd   # gradient at x
    H = p.hes   # Hessian at x
    # Keeps track of the number of iterations
    newt_res = minimizenewt(f,g,H,x0)
    BFGW_res = minimizeBFGS(f,g,eye(length(x0)), x0)

    # NM = Newton's method
    # NMNPDF = Newton's method with non-positive definite fix
    @printf("%3i  %-31s %-6i %-20f %-8i %-20f \n",
            i, 
            p.name,
            newt_res[2],
            newt_res[1],
            BFGW_res[2],
            BFGW_res[1])
end


println("done")

Legends:
NM = Newton's method
BFGS = quasi-Newton method using BFGS approximation of inverse Hessian
---------------------------------------------------------------------------------------
                                     # of iterations (-1 = failure)
No.  Name                            NW     value                BFGS     value                
  1  Hellical valley                 1      24.781713            6        10.239900            
  2  Bigg's EXP6                     500    0.779070             15       0.014532             
  3  Gaussian                        1      0.000000             2        0.000000             
  4  Powell                          500    1.135262             8        0.020217             
  5  Box 3-dim                       2      0.132361             7        0.507272             
  6  Variably dimensioned            6      115248628.457825     1        167122090.748794     
  7  Watson                          500    12.702530            4     

In [6]:
i = 1
p = Problem(i)
x0 = p.x0       # standard starting point
f = p.obj   # objective value at x
g = p.grd   # gradient at x
H = p.hes   # Hessian at x

minimizenewt(f,g,H,x0);
minimizeBFGS(f,g,eye(length(x0)), x0)

5-element Array{Float64,1}:
 -0.40318 
  0.922119
  3.18562 
 10.2399  
  6.0     

2  Bigg's EXP6                     -1     15       
done
